In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from torch import autograd
from torch.autograd import Variable

In [2]:
reading = np.load('reading_np.npy')
phn = np.load('phone_np.npy')
drinking = np.load('drinking_np.npy')

In [3]:
y_reading = np.full(50,0)
y_phn = np.full(50,1)
y_drink = np.full(50,2)
Y = np.concatenate((y_reading, y_phn, y_drink), axis = 0)

In [4]:
N = Y.shape[0]

N_actions = 3
N_features = 16380
dtype = torch.FloatTensor

In [5]:
one_hot_labels = np.zeros((N, N_actions))
one_hot_labels[np.arange(N), Y] = 1

In [6]:
X = np.vstack((phn,reading,drinking))


In [7]:
train_x = torch.from_numpy(X)

In [8]:
ffw_train = train_x.view(train_x.shape[0],-1)
print(ffw_train.shape)

mean = ffw_train.mean(dim=0)
std = ffw_train.std(dim=0)

ffw_train = (ffw_train - mean)/std

torch.Size([150, 16380])


In [9]:
ffw_train = ffw_train.float()

In [10]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(N_features,2048)
        self.fc2 = nn.Linear(2048,512)
        self.fc3 = nn.Linear(512,N_actions)
        
    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.tanh(self.fc3(x))
        #print(x.shape)
        sft = nn.Softmax(dim=1)
        x = sft(x)
        return x

net = Net()
print(net)

Net(
  (fc1): Linear(in_features=16380, out_features=2048)
  (fc2): Linear(in_features=2048, out_features=512)
  (fc3): Linear(in_features=512, out_features=3)
)


In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr = 0.01, momentum=0.9)

In [12]:
train_x = Variable(ffw_train).float()
labels = Variable(torch.from_numpy(Y))
#labels = Variable(torch.from_numpy(one_hot_labels))

In [13]:
net = net.cuda()

In [14]:
num_epochs = 100
for epoch in range(num_epochs):
        # Convert torch tensor to Variable
        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer
        outputs = net(train_x.cuda())
        outputs = outputs
        loss = criterion(outputs, labels.cuda())
        loss.backward()
        optimizer.step()
        
        print ('Epoch [%d/%d], Loss: %.4f' %(epoch+1, num_epochs, loss.data[0]))

Epoch [1/100], Loss: 1.1009
Epoch [2/100], Loss: 1.0008
Epoch [3/100], Loss: 0.9172
Epoch [4/100], Loss: 0.8852
Epoch [5/100], Loss: 0.8678
Epoch [6/100], Loss: 0.8547
Epoch [7/100], Loss: 0.8436
Epoch [8/100], Loss: 0.8340
Epoch [9/100], Loss: 0.8254
Epoch [10/100], Loss: 0.8177
Epoch [11/100], Loss: 0.8108
Epoch [12/100], Loss: 0.8045
Epoch [13/100], Loss: 0.7985
Epoch [14/100], Loss: 0.7928
Epoch [15/100], Loss: 0.7872
Epoch [16/100], Loss: 0.7818
Epoch [17/100], Loss: 0.7768
Epoch [18/100], Loss: 0.7722
Epoch [19/100], Loss: 0.7680
Epoch [20/100], Loss: 0.7642
Epoch [21/100], Loss: 0.7606
Epoch [22/100], Loss: 0.7574
Epoch [23/100], Loss: 0.7544
Epoch [24/100], Loss: 0.7518
Epoch [25/100], Loss: 0.7494
Epoch [26/100], Loss: 0.7472
Epoch [27/100], Loss: 0.7451
Epoch [28/100], Loss: 0.7432
Epoch [29/100], Loss: 0.7414
Epoch [30/100], Loss: 0.7398
Epoch [31/100], Loss: 0.7382
Epoch [32/100], Loss: 0.7368
Epoch [33/100], Loss: 0.7355
Epoch [34/100], Loss: 0.7344
Epoch [35/100], Loss: 0

In [44]:
def pred(x):
    out = net(x)
    weight, action = torch.max(out, dim=1)
    return action.data.cpu().numpy()

In [50]:
def accuracy(x,labels):
    N = labels.shape[0]
    predicted = pred(x.cuda())
    i = 0
    for yhat,y in zip(predicted,labels):
        
        if yhat == y:
            i += 1
    print('Accuracy:', i*100/N)
        
    

In [53]:
accuracy(train_x, Y)

Accuracy: 98.66666666666667
